In [9]:
# package import

import os
import numpy as np
from scipy.io import loadmat
import matplotlib as mpl
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import itertools
import math
import sklearn as sk
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import seaborn as sns

# set wd

os.chdir("\\Users\\madel\\Documents\\Grad School\\Classes\\Fall 2020\\ECE532\\Final Project")
os.getcwd() # sets wd
#os.listdir(os.getcwd()) # lists files in wd


'C:\\Users\\madel\\Documents\\Grad School\\Classes\\Fall 2020\\ECE532\\Final Project'

In [10]:
# import data
reddf = pd.read_csv("winequality-red.csv",delimiter=';')
whitedf = pd.read_csv("winequality-white.csv",delimiter=';')

# add color label
reddf['color'] = str('red')
whitedf['color']=str('white')
    
# stack df's 
fulldf = pd.concat([reddf, whitedf], ignore_index=True)
#fulldf.head()
#fulldf.tail()

In [ ]:
### Hypothesis: Including all features will lead to noisy clustering 
### and a higher error rate than excluding some features

In [3]:
### 1. All Data ###

# remove color for data formatting
trimdf = fulldf.drop('color',axis=1)

In [4]:
# scale and transform the data

# since kmeans clustering doesn't handle irregularly shaped data
# each parameter will be scaled between 0 and 1
X=trimdf
scaler = MinMaxScaler() 
scaler.fit(X)
scaleddf=pd.DataFrame(scaler.transform(X))

In [5]:
# run kmeans on scaled df
kmeans = KMeans(n_clusters=2, init='random', n_init=50, random_state=0).fit(scaleddf)
print('Cluster Centers: ' , str(kmeans.cluster_centers_))
print('SS distances: ' + str(kmeans.inertia_))
print('Number of Iterations: ' + str(kmeans.n_iter_))

Cluster Centers:  [[0.29206849 0.18481443 0.18996717 0.09349989 0.08957278 0.10941891
  0.28034535 0.17506207 0.37345947 0.17378814 0.24537775 0.40030422]
 [0.26812159 0.15624969 0.19480099 0.04659765 0.06164338 0.09257737
  0.21328209 0.10477242 0.40512758 0.17642813 0.52788939 0.56974706]]
SS distances: 699.4912445148213
Number of Iterations: 8


In [6]:
# add kmeans cluster labels to df

labels = kmeans.labels_
labels = labels.tolist()
fulldf['label'] = labels
#fulldf.head()

In [7]:
## we'll check for errors by comapring 'color' and 'label' columns ##
#  assume Red = 1 #
correct = []
incorrect = []
for z in list(fulldf.index.values):
    if fulldf.at[z, 'color'] == 'red' and fulldf.at[z, 'label']==1:
        correct.append(z)
    elif fulldf.at[z,'color']=='white' and fulldf.at[z,'label']==1:
        incorrect.append(z)
    elif fulldf.at[z, 'color'] == 'red' and fulldf.at[z, 'label']==0:
        incorrect.append(z)
    elif fulldf.at[z,'color']=='white' and fulldf.at[z,'label']==0:
        correct.append(z)
        
n_total = len(fulldf.index.values)
n_errors = len(incorrect)/n_total
print('Full Data Error Rate: ' + str(n_errors*100) + '%')

Full Data Error Rate: 47.17561951670002%


In [11]:
### 2. Remove Some Features ###
# alcohol content and rating show similar distributions 
# between red and white, and might not be predictive

# remove quality and alcohol content
trimdf = fulldf.drop('color',axis=1)
trimdf = trimdf.drop('quality',axis=1)
trimdf = trimdf.drop('alcohol',axis=1)
#trimdf.head()

In [12]:
# scale and transform the data

# since kmeans clustering doesn't handle irregularly shaped data
# each parameter will be scaled between 0 and 1
X=trimdf
scaler = MinMaxScaler() 
scaler.fit(X)
scaleddf=pd.DataFrame(scaler.transform(X))

In [13]:
# run kmeans on scaled df
kmeans = KMeans(n_clusters=2, init='random', n_init=50, random_state=0).fit(scaleddf)
print('Cluster Centers: ' , str(kmeans.cluster_centers_))
print('SS distances: ' + str(kmeans.inertia_))
print('Number of Iterations: ' + str(kmeans.n_iter_))

Cluster Centers:  [[0.25302069 0.1295245  0.20279466 0.08930091 0.06146042 0.1202644
  0.30717106 0.13376085 0.36115965 0.151668  ]
 [0.36793108 0.30083686 0.16016152 0.03027309 0.12697642 0.05051527
  0.09373545 0.18289756 0.4605017  0.24286112]]
SS distances: 410.2867585703211
Number of Iterations: 5


In [14]:
# add kmeans cluster labels to df

labels = kmeans.labels_
labels = labels.tolist()
fulldf['label'] = labels
#fulldf.head()

In [15]:
# we'll check for errors by comapring 'color' and 'label' columns
# assume Red = 1 #
correct = []
incorrect = []
for z in list(fulldf.index.values):
    if fulldf.at[z, 'color'] == 'red' and fulldf.at[z, 'label']==1:
        correct.append(z)
    elif fulldf.at[z,'color']=='white' and fulldf.at[z,'label']==1:
        incorrect.append(z)
    elif fulldf.at[z, 'color'] == 'red' and fulldf.at[z, 'label']==0:
        incorrect.append(z)
    elif fulldf.at[z,'color']=='white' and fulldf.at[z,'label']==0:
        correct.append(z)
        
n_total = len(fulldf.index.values)
n_errors = len(incorrect)/n_total
print('Trimmed Data Error Rate: ' + str(n_errors*100) + '%')

Trimmed Data Error Rate: 1.970140064645221%
